In [1]:
%load_ext autoreload
%autoreload 2

import util as yu
from util import *
import util_moments as yum
from scipy.integrate import quad

yu.setpath('test_lhapdf')

if "_has_run_once" not in globals():
    import os, sys, ctypes
    _has_run_once = True
    sys.path.insert(0, "/p/project1/ngff/li47/code/LHAPDF/build/lib/python3.13/site-packages")
    ctypes.CDLL("/p/project1/ngff/li47/code/LHAPDF/build/lib/libLHAPDF.so", mode=ctypes.RTLD_GLOBAL)
    import lhapdf

In [2]:
lhapdf.availablePDFSets()

['NNPDF31_nnlo_as_0118',
 'NNPDF40_an3lo_as_01180',
 'NNPDF40_nlo_as_01180',
 'NNPDF40_nnlo_as_01180',
 'NNPDF40_nnlo_as_01180_qed']

In [20]:
PDF_SET      = "NNPDF40_nnlo_as_01180"
pdfset = lhapdf.getPDFSet(PDF_SET)
replicas = pdfset.mkPDFs()

LHAPDF 6.5.5 loading all 101 PDFs in set NNPDF40_nnlo_as_01180
NNPDF40_nnlo_as_01180, version 1; 101 PDF members


In [21]:
FLAVORS = {
    "u+":  (2, -2),
    "d+":  (1, -1),
    "s+":  (3, -3),
    "c+": (4,-4),
    # "b+": (5,-5),
    # "t+": (6,-6),
    "g":   (21,),
    # 'gm': (22,),
}
Q=2.0

def pdf_moment(pdf, xs, Q, n):
    func = lambda x: x**(n-1) * sum(pdf.xfxQ(fl, x, Q) for fl in xs)
    val, _ = quad(func, 1e-6, 1.0, epsabs=1e-6, epsrel=1e-4)
    return val

# Prepare storage
results = {name: [] for name in FLAVORS}

# Loop replicas
for i, pdf in enumerate(replicas):
    for name, fls in FLAVORS.items():
        m = pdf_moment(pdf, fls, Q, n=1)  # first moment <x>
        results[name].append(m)

In [22]:
print(f"PDF moments at Q={Q} GeV (NNPDF3.1 NNLO, {len(replicas)} replicas):")
s=0
for name, vals in results.items():
    arr = np.array(vals)
    mean = arr.mean()
    s+=mean
    std  = arr.std(ddof=1)        # Monte Carlo uncertainty
    print(f"  {name}: {mean:.5f} ± {std:.5f}  (68% CL)")
    
s

PDF moments at Q=2.0 GeV (NNPDF3.1 NNLO, 101 replicas):
  u+: 0.34675 ± 0.00263  (68% CL)
  d+: 0.19345 ± 0.00323  (68% CL)
  s+: 0.04834 ± 0.00338  (68% CL)
  c+: 0.01195 ± 0.00221  (68% CL)
  g: 0.39951 ± 0.00260  (68% CL)


0.9999986427038596